In [36]:
import os
import cv2
import torch
import numpy as np
from torchvision.models import resnet50
import torchvision.transforms as transforms
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import torchvision.transforms as transforms
from skimage.feature import local_binary_pattern
import pickle

In [37]:
def preprocessing_and_feature_extraction(flooded_image_folder):
    # Function to read and preprocess the images
    def preprocess_image(image_path):
    # Read the image
        image = cv2.imread(image_path)
        
        # Apply preprocessing steps
        # For example, you can resize the image to a specific size (e.g., 256x256)
        resized_image = cv2.resize(image, (256, 256))
        
        # You can apply additional preprocessing steps here, such as noise removal, image enhancement, etc.
        
        # Normalize the pixel values to a specific range (e.g., 0-1)
        normalized_image = resized_image.astype(np.float32) / 255.0
        
        return normalized_image
    
    # Read and preprocess the flooded images
    flooded_images = []
    for image_file in os.listdir(flooded_image_folder):
        image_path = os.path.join(flooded_image_folder, image_file)
        preprocessed_image = preprocess_image(image_path)
        flooded_images.append(preprocessed_image)

    # Convert the image lists to NumPy arrays
    flooded_images = np.array(flooded_images)
    # Convert the image lists to NumPy arrays
    print("Shape of flooded images array:", flooded_images.shape)


    def extract_cnn_features(image):
        # Preprocess the image for ResNet-50
        transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
        # Convert the image to uint8 data type
        image = (image * 255).astype(np.uint8)
        
        # Apply the transformations
        preprocessed_image = transform(image)
        
        # Load the pre-trained ResNet-50 model
        model = resnet50(pretrained=True)
        
        # Remove the last fully connected layer from the model
        features_model = torch.nn.Sequential(*list(model.children())[:-1])
        
        # Set the model to evaluation mode
        features_model.eval()
        
        # Extract features from the preprocessed image
        with torch.no_grad():
            features = features_model(preprocessed_image.unsqueeze(0))
        
        # Flatten the features
        flattened_features = features.view(features.size(0), -1).numpy()
        
        return flattened_features
    
    # Function to extract features using Local Binary Patterns (LBP)
    def extract_lbp_features(image):
        # Convert the image to grayscale
        grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Compute LBP features
        lbp = local_binary_pattern(grayscale_image, 8, 1, method='uniform')
        histogram, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 10), range=(0, 10))
        normalized_histogram = histogram.astype(np.float32) / np.sum(histogram)
    
        return normalized_histogram
    



    # Apply feature extraction methods to the flooded images
    flooded_cnn_features = []
    flooded_lbp_features = []

    for image in flooded_images:
        flooded_cnn_features.append(extract_cnn_features(image))
        flooded_lbp_features.append(extract_lbp_features(image))


    # Convert the feature lists to NumPy arrays
    flooded_cnn_features = np.array(flooded_cnn_features)
    flooded_lbp_features = np.array(flooded_lbp_features)

    # Print the shapes of the feature arrays
    print("Shape of flooded CNN features array:", flooded_cnn_features.shape)
    print("Shape of flooded lbp features array:", flooded_lbp_features.shape)

    return flooded_cnn_features , flooded_lbp_features




In [38]:
for image_file in sorted(os.listdir("test/")):
    image_path = os.path.join("test/", image_file)
    print(image_path)

test/12.jpg
test/13.jpg
test/14.jpg
test/15.jpg
test/16.jpg
test/17.jpg
test/18.jpg
test/19.jpg
test/20.jpg
test/21.jpg
test/22.jpg
test/23.jpg
test/24.jpg
test/25.jpg
test/26.jpg
test/27.jpg
test/28.jpg
test/29.jpg
test/30.jpg
test/31.jpg
test/32.jpg
test/33.jpg
test/34.jpg
test/35.jpg
test/36.jpg
test/37.jpg
test/38.jpg
test/39.jpg
test/40.jpg
test/41.jpg
test/42.jpg
test/43.jpg
test/44.jpg
test/45.jpg
test/46.jpg
test/47.jpg
test/602.jpg
test/603.jpg
test/604.jpg
test/605.jpg
test/606.jpg
test/607.jpg
test/608.jpg
test/609.jpg
test/610.jpg
test/611.jpg
test/612.jpg
test/613.jpg
test/614.jpg
test/615.jpg
test/616.jpg
test/617.jpg
test/618.jpg
test/619.jpg
test/620.jpg
test/621.jpg
test/622.jpg
test/623.jpg
test/624.jpg
test/625.jpg
test/626.jpg
test/627.jpg
test/628.jpg
test/629.jpg
test/630.jpg
test/631.jpg
test/632.jpg
test/633.jpg
test/634.jpg
test/635.jpg
test/636.jpg
test/637.jpg


In [39]:
def random_forest_classification (cnn_features , lbp_features):
    # Reshape the features array to 2D
    cnn_features = cnn_features.reshape(cnn_features.shape[0], -1)
    features = features = np.concatenate((cnn_features, lbp_features), axis=1)

    # Combine the labels for training


    rf_classifier = pickle.load(open("model.pkl", 'rb'))

    # Make predictions on the test set
    y_pred = rf_classifier.predict(features)

    
    return y_pred


In [41]:
test_fiel_path = "test/"
cnn_features , lbp_features = preprocessing_and_feature_extraction(test_fiel_path)
prediction = random_forest_classification(cnn_features,lbp_features) 

Shape of flooded images array: (72, 256, 256, 3)


c:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Shape of flooded CNN features array: (72, 1, 2048)
Shape of flooded lbp features array: (72, 9)


In [45]:
with open("pred.txt", 'w') as file:
    # Write the list elements to the file
    for item in prediction:
        file.write(str(int(item)) + '\n')

In [43]:
with open("true.txt", 'w') as file:
    # Write the list elements to the file
    for i in range(36):
        file.write("1" + '\n')
    for i in range(36):
        file.write("0" + '\n')